<a href="https://colab.research.google.com/github/dogee4803/db_course/blob/main/DB_Week13_SQL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 1: Efficient Indexing

In [ ]:
import sqlite3 as sl
# Connect to SQLite database
conn = sl.connect('example.db')
c = conn.cursor()

In [ ]:
# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS employees(
              id INTEGER PRIMARY KEY,
              name TEXT,
              department TEXT,
              salary INTEGER
              )'''
              )

In [3]:
# This example demonstrates how to create an index on the 'department' column.
# The index will improve the perfomance of queries filtering by department

# Adding some sample data
sample_data = [(1, 'Alice', 'HR', 7000), (2, 'Bob', 'Engineering', 8000), (3, 'Charlie', 'HR', 75000)]
c.executemany('INSERT INTO employees VALUES (?,?,?,?)', sample_data)

# Creating an index
c.execute('CREATE INDEX idx_department ON employees (department)')

# Indexes can significantly improve the perfomance of quries that filter or sort based on the indexed column.
# However, they can also slow down data  insertation and updates

# Querying with and without the index
c.execute('SELECT * FROM employees WHERE department = "HR"')
print("Query plan with index: ", c.fetchall())

# Drop the index
c.execute('DROP INDEX idx_department')

c.execute('SELECT * FROM employees WHERE department = "HR"')
print("Query plan without index: ", c.fetchall())

# Cleanup
conn.commit()
conn.close()

Query plan with index:  [(1, 'Alice', 'HR', 7000), (3, 'Charlie', 'HR', 75000)]
Query plan without index:  [(1, 'Alice', 'HR', 7000), (3, 'Charlie', 'HR', 75000)]


# Example 2: Query optomization example

In [4]:
import sqlite3 as sl
# Connect to SQLite database
conn = sl.connect('example1.db')
c = conn.cursor()

In [5]:
# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS products(
              id INTEGER PRIMARY KEY,
              name TEXT,
              category TEXT,
              price INTEGER
              )'''
              )

In [12]:
# This example demonstrates how to write an optimized query by selecting only the necessary columns
# and using efficient filtering.

# Adding some sample data
sample_data = [(1, 'Laptop', 'Electronics', 1200), (2, 'Chair', 'Furniture', 150), (3, 'Smartphone', 'Electronics', 800)]
c.executemany('INSERT INTO products VALUES (?,?,?,?)', sample_data)

# Selecting specific columns and filtering
c.execute('SELECT name, price FROM products WHERE category = "Electronics"')
print("Efficient query results: ", c.fetchall())

# Cleanup
conn.commit()
conn.close()

Efficient query results:  [('Laptop', 1200), ('Smartphone', 800), ('Laptop', 1200), ('Smartphone', 800)]


# Example 3: Transaction optimization

In [13]:
import sqlite3 as sl
# Connect to SQLite database
conn = sl.connect('example2.db')
c = conn.cursor()

In [14]:
# Create a sample table
c.execute('''CREATE TABLE IF NOT EXISTS orders(
              id INTEGER PRIMARY KEY,
              product_id INTEGER,
              quantity INTEGER
              )'''
              )

In [15]:
# This example demonstrates the use of transactions to optimize bulk data insertations.

# Start a transaction
c.execute('BEGIN TRANSACTION')

# Bulk insert data
bulk_data = [(i, i % 5 + 1, i * 2) for i in range(1, 1001)]
c.executemany('INSERT INTO orders VALUES (?,?,?)', bulk_data)

# Commit the transaction
conn.commit()

# Query to verify data insertion
c.execute('SELECT COUNT(*) FROM orders')
print("Number of records inserted: ", c.fetchone()[0])

# Cleanup
conn.close()

Number of records inserted:  1000


# Example 4: Efficient join query

In [16]:
import sqlite3 as sl
# Connect to SQLite database
conn = sl.connect('example3.db')
c = conn.cursor()

In [17]:
# Create sample tables
c.execute('''CREATE TABLE IF NOT EXISTS students(
              id INTEGER PRIMARY KEY,
              name TEXT,
              class_id INTEGER
              )'''
              )
c.execute('''CREATE TABLE IF NOT EXISTS classes(
              id INTEGER PRIMARY KEY,
              class_name TEXT
              )'''
              )

In [18]:
# Adding some sample data
students_data = [(1, 'John', 1), (2, 'Emma', 2), (3, 'Harry', 1)]
classes_data = [(1, 'Math'), (2, 'Science')]
c.executemany('INSERT INTO students VALUES (?,?,?)', students_data)
c.executemany('INSERT INTO classes VALUES (?,?)', classes_data)

In [19]:
# This example demonstrates how to perform an efficient join between two tables.
c.execute('''SELECT students.name, classes.class_name
          FROM students
          JOIN classes ON students.class_id = classes.id''')
print("Join results: ", c.fetchall())

# Cleanup
conn.commit()
conn.close()

Join results:  [('John', 'Math'), ('Emma', 'Science'), ('Harry', 'Math')]


# Example 5: Advanced PRAGMA Optimization Example

In [1]:
import sqlite3 as sl
# Connect to SQLite database
conn = sl.connect('example4.db')
c = conn.cursor()

In [2]:
# This example demonstrates the advanced use of PRAGMA statements
# for various database optimizations including write-ahead logging, foreign key enforcement,
# and query perfomance improvement

# 1. Setting the journal mode to WAL (Write-Ahead Logging)
# WAL can improve the perfomance of read operations write operations are happening.
c.execute('PRAGMA journal_mode=WAL')
journal_mode = c.fetchone()[0]
print("Journal mode set to: ", journal_mode)

# 2. Enabling foreign key constraints
# Ensures referential integrity in the database.
c.execute('PRAGMA foreign_keys=ON')
c.execute('PRAGMA foreign_keys')
fk_status = c.fetchone()[0]
print("Foreign keys status:", "Enabled" if fk_status else "Disabled")

# 3. Setting synchronous mode to NORMAL
# This reduces the number of disk writes, speeding up transactions at the risk of some data loss in case of crash.
c.execute('PRAGMA synchronous=NORMAL')
c.execute('PRAGMA synchronous')
sync_mode = c.fetchone()[0]
print("Synchronous mode set to: ", sync_mode)

# 4. Adjusting cache size
# Increasing the cache size can improve the database read perfomance by reducing disk 1/0
c.execute('PRAGMA cache_size=10000') # Setting cache size to 10000 pages
c.execute('PRAGMA cache_size')
cache_size = c.fetchone()[0]
print("Cache size set to: ", cache_size, "pages")

# 5. Querying the database file size
# Useful for monitoring the database size, which can be a factor in perfomance.
c.execute('PRAGMA page_size')
page_size = c.fetchone()[0]
c.execute('PRAGMA page_count')
page_count = c.fetchone()[0]
db_size = page_size * page_count
print("Database size: ", db_size, "bytes")

# Cleanup
conn.close()

Journal mode set to:  wal
Foreign keys status: Enabled
Synchronous mode set to:  1
Cache size set to:  10000 pages
Database size:  4096 bytes
